# Импорты

In [1]:
import os
import sys

# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [2]:
# Импортируем функции предобработки из utils
from utils import preprocess_functions as ppf
from utils import metrics as mtc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\filat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\filat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import re


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import joblib
import itertools

# Загружаем разделенные данные

In [4]:
reference = pd.read_csv("../../data/reference.csv")
train = pd.read_csv("../../data/splited/train.csv")
valid = pd.read_csv("../../data/splited/valid.csv")
test = pd.read_csv("../../data/splited/test.csv")

In [5]:
reference.head(1)

,id,name,region
0,69,Кировска,Мурманская область


In [6]:
train.head(1)

,name,school_id
0,"ХМАО-Югра, СШ г. Покачи",173


# Обработка reference.region

In [7]:
reference.region = reference.region.apply(ppf.simple_preprocess_text).str.lower()

In [8]:
region_dict = joblib.load("../../resources/general/region_dict.joblib")

In [9]:
for reference_region in reference.region.sort_values().unique():
    if reference_region not in region_dict:
        for region in region_dict:
            if reference_region in region_dict[region]:
                reference.region = reference.region.str.replace(
                    f"{reference_region}", f"{region}"
                )
                print(f"{reference_region} -> {region}")

набережные челны -> республика татарстан
северодвинск -> архангельская область


In [10]:
for reference_region in reference.region.sort_values().unique():
    if reference_region not in region_dict:
        print(f"Unknown region: {reference_region}")

Unknown region: республика саха
Unknown region: республика чувашия
Unknown region: хмао югра
Unknown region: ямало ненецкий ао


In [11]:
reference.region = reference.region.str.replace(
    "республика саха",
    "республика саха якутия",
)
reference.region = reference.region.str.replace(
    "республика чувашия",
    "чувашская республика",
)
reference.region = reference.region.str.replace(
    "хмао югра",
    "ханты мансийский автономный округ",
)
reference.region = reference.region.str.replace(
    "ямало ненецкий ао",
    "ямало ненецкий автономный округ",
)

In [12]:
reference.head()

,id,name,region
0,69,Кировска,мурманская область
1,70,Клин спортивный,московская область
2,71,КО СШ по ЗВС,калининградская область
3,8,Айсберг,свердловская область
4,73,Команда Дмитриева,челябинская область


Удалим дубликаты

In [13]:
reference = reference[~(reference.id == 296)]

In [14]:
reference.to_csv("../../data/preprocessed/general/reference.csv", index=False)

# Простая обработка текста

In [15]:
reference["new"] = reference.name.apply(ppf.simple_preprocess_text).str.lower()
reference.new = reference.new.apply(ppf.replace_numbers_with_text)

In [16]:
reference.loc[[187, 51], :]

,id,name,region,new
187,206,СШОР No 1,самарская область,сшор один
51,53,ДЮСШ No 1,тюменская область,дюсш один


# Раширенная обработка текста: работа с аббревиатурами

## Удаление всех аббревиатур

Формируем blacklist и whitelist аббревиатур, для этого не обязательно иметь аббревиатуры в верхнем регистре. Аббериватуры из blacklist будут удалены.

In [17]:
abbr_dict = joblib.load("../../resources/general/abbreviations_dict.joblib")

Расшифровываем все аббревиатуры

In [18]:
def preprocess_text(
    name,
    abbreviation_dict,
    output_list=False,
    unknown_answer=False,
    remove_unknown_abbr=False,
    remove_all_abbr=False,
):
    two_letter_prepositions = [
        " в ",
        " во ",
        " до ",
        " из ",
        " на ",
        " по ",
        " о ",
        " об ",
        " обо ",
        " у ",
    ]

    symbols = [" no", " NO", " No", "номер"]

    # Удаляем служебные символы (перенос строки, табуляция и т.д.)
    name = re.sub(r"[\n\t\r]", " ", name)

    # Создаем регулярное выражение для предлогов
    prepositions_pattern = (
        r"\b(?:" + "".join(two_letter_prepositions) + "".join(symbols) + r")\b"
    )

    # Заменяем все предлоги на пробем (предварительное решение вместо трудоемкого удаления стоп-слов)
    name = re.sub(prepositions_pattern, " ", name)

    # Удаление пунктуации
    name = re.sub(r"[^\w\s]", " ", name)

    # Удаление отдельных букв
    name = re.sub(r"\b[А-ЯЁа-яё]\b", " ", name)

    # Удаление букв ё
    name = re.sub(r"[Ёё]", "е", name)

    unknown_abbr = []
    # Находим аббревиатуры большими буквами и приводим их к нижнему регистру
    # Надо уточнить поиск неизвестных.
    # А если в конце аббревиатуры прописная буква?
    uppercase_abbreviations = re.findall(r"\b[А-ЯЁ]+[а-яё]*+[А-ЯЁ]+\b", name)
    for abbr in uppercase_abbreviations:
        abbr = abbr.lower()
        if abbr not in abbreviation_dict:
            unknown_abbr.append(abbr.upper())
            if remove_unknown_abbr:
                name = re.sub(r"\b" + re.escape(abbr.upper()) + r"\b", " ", name)

    # Удаление лишних пробелов
    name = re.sub(r"\s+", " ", name)

    # Удаление пробелов в начале и в конце
    name = name.strip()

    possible_replacements = []
    parts = name.lower().split()

    for part in parts:
        if part in abbreviation_dict:
            if not remove_all_abbr:
                replacements = abbreviation_dict[part]
                if isinstance(replacements, str):
                    replacements = [replacements]
                elif not output_list:
                    replacements = [""]
                possible_replacements.append(replacements)
            else:
                pass
        else:
            possible_replacements.append([part])

    # Генерируем все возможные комбинации
    all_combinations = list(itertools.product(*possible_replacements))

    # Формируем итоговые наименования
    final_phrases = [" ".join(combination).strip() for combination in all_combinations]

    if not output_list:
        final_phrases = final_phrases[0]

    if unknown_answer:
        return list(set(unknown_abbr))

    return final_phrases

In [19]:
reference["extended_process"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.extended_process = reference.extended_process.apply(
    ppf.replace_numbers_with_text
)
reference.extended_process = reference.extended_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, True)
)

In [20]:
reference.loc[[187, 51], :]

,id,name,region,new,extended_process
187,206,СШОР No 1,самарская область,сшор один,один
51,53,ДЮСШ No 1,тюменская область,дюсш один,один


## Расшифровка аббревиатур, удаление неизвестных аббревиатур

In [21]:
reference["extended2_process"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.extended2_process = reference.extended2_process.apply(
    ppf.replace_numbers_with_text
)
reference.extended2_process = reference.extended2_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)

In [22]:
reference.loc[[187, 51], :]

,id,name,region,new,extended_process,extended2_process
187,206,СШОР No 1,самарская область,сшор один,один,спортивная школа олимпийского резерва один
51,53,ДЮСШ No 1,тюменская область,дюсш один,один,детско юношеская спортивная школа один


In [23]:
train[train.school_id == 217][["name", "school_id"]]

,name,school_id


# Работа с регионами

## Расшифровка аббревиатур, удаление неизвестных аббревиатур, выделение регионов

In [24]:
train["region_process"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.region_process = train.region_process.apply(ppf.replace_numbers_with_text)
train.region_process = train.region_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)

In [25]:
train["region"] = train.region_process.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

train.region_process = train.region_process.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)

In [26]:
train.head(15)

,name,school_id,region_process,region
0,"ХМАО-Югра, СШ г. Покачи",173,спортивная школа покачи,ханты мансийский автономный округ
1,"Санкт-Петербург, СПб ГБУ СШОР по фигурному кат...",198,государственное бюджетное учреждение спортивна...,санкт петербург
2,"ЯНАО, ГАУ ДО ЯНАО СШ ""Ямал""",267,государственное автономное учреждение дополнит...,ямало ненецкий автономный округ
3,"МО Смольнинское, СФК Ice Dream ИП Карлинской",5,смольнинское секция фигурного катания ice drea...,None
4,"Москва, МАФК школа ""Олимп""",119,московская академия фигурного катания на коньк...,москва
5,"Пермский край, СШОР ""Старт""",168,спортивная школа олимпийского резерва старт,пермский край
6,"Московская область, МБУС ""Одинцово""",116,муниципальное бюджетное учреждение спорта один...,московская область
7,Айсдрим,5,айсдрим,None
8,"Свердловская область, МБУ СШ №8 ""Локомотив"", г...",90,муниципальное бюджетное учреждение спортивная ...,свердловская область
9,"Республика Татарстан, ГБУ РСШОР по ФФК и ШТ",201,государственное бюджетное учреждение республик...,республика татарстан


## Удаление организационно-правовых форм

Создание blacklist ОПФ

In [27]:
blacklist_opf = joblib.load("../../resources/general/blacklist_opf.joblib")


In [28]:
def find_substrings(input_list):
    problem_strings = []
    for i in range(len(input_list)):
        current_string = input_list[i]
        for j in range(i + 1, len(input_list)):
            next_string = input_list[j]
            if current_string in next_string:
                problem_strings.append(current_string)
                break  # прекращаем проверку текущей строки, если она уже найдена как подстрока
    return problem_strings


In [29]:
find_substrings(blacklist_opf)

[]

In [30]:
train["no_opf_process"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.no_opf_process = train.no_opf_process.apply(ppf.replace_numbers_with_text)
train.no_opf_process = train.no_opf_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)


In [31]:
train["region"] = train.no_opf_process.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

train.no_opf_process = train.no_opf_process.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)


In [32]:
train.no_opf_process = train.no_opf_process.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)


In [33]:
train.head(15)

,name,school_id,region_process,region,no_opf_process
0,"ХМАО-Югра, СШ г. Покачи",173,спортивная школа покачи,ханты мансийский автономный округ,спортивная школа покачи
1,"Санкт-Петербург, СПб ГБУ СШОР по фигурному кат...",198,государственное бюджетное учреждение спортивна...,санкт петербург,спортивная школа олимпийского резерва по фигур...
2,"ЯНАО, ГАУ ДО ЯНАО СШ ""Ямал""",267,государственное автономное учреждение дополнит...,ямало ненецкий автономный округ,спортивная школа ямал
3,"МО Смольнинское, СФК Ice Dream ИП Карлинской",5,смольнинское секция фигурного катания ice drea...,None,смольнинское секция фигурного катания ice drea...
4,"Москва, МАФК школа ""Олимп""",119,московская академия фигурного катания на коньк...,москва,московская академия фигурного катания на коньк...
5,"Пермский край, СШОР ""Старт""",168,спортивная школа олимпийского резерва старт,пермский край,спортивная школа олимпийского резерва старт
6,"Московская область, МБУС ""Одинцово""",116,муниципальное бюджетное учреждение спорта один...,московская область,одинцово
7,Айсдрим,5,айсдрим,None,айсдрим
8,"Свердловская область, МБУ СШ №8 ""Локомотив"", г...",90,муниципальное бюджетное учреждение спортивная ...,свердловская область,спортивная школа восемь локомотив екатеринбург
9,"Республика Татарстан, ГБУ РСШОР по ФФК и ШТ",201,государственное бюджетное учреждение республик...,республика татарстан,республиканская спортивная школа олимпийского ...


## Удаление городов

In [34]:
train["no_city_process"] = train.no_opf_process.apply(
    ppf.process_cities, args=(region_dict,)
)


In [35]:
train[["name", "school_id", "region_process", "region", "no_city_process"]].head(50)

,name,school_id,region_process,region,no_city_process
0,"ХМАО-Югра, СШ г. Покачи",173,спортивная школа покачи,ханты мансийский автономный округ,спортивная школа
1,"Санкт-Петербург, СПб ГБУ СШОР по фигурному кат...",198,государственное бюджетное учреждение спортивна...,санкт петербург,спортивная школа олимпийского резерва по фигур...
2,"ЯНАО, ГАУ ДО ЯНАО СШ ""Ямал""",267,государственное автономное учреждение дополнит...,ямало ненецкий автономный округ,спортивная школа ямал
3,"МО Смольнинское, СФК Ice Dream ИП Карлинской",5,смольнинское секция фигурного катания ice drea...,None,смольнинское секция фигурного катания ice drea...
4,"Москва, МАФК школа ""Олимп""",119,московская академия фигурного катания на коньк...,москва,московская академия фигурного катания на коньк...
5,"Пермский край, СШОР ""Старт""",168,спортивная школа олимпийского резерва старт,пермский край,спортивная школа олимпийского резерва старт
6,"Московская область, МБУС ""Одинцово""",116,муниципальное бюджетное учреждение спорта один...,московская область,
7,Айсдрим,5,айсдрим,None,айсдрим
8,"Свердловская область, МБУ СШ №8 ""Локомотив"", г...",90,муниципальное бюджетное учреждение спортивная ...,свердловская область,спортивная школа восемь локомотив
9,"Республика Татарстан, ГБУ РСШОР по ФФК и ШТ",201,государственное бюджетное учреждение республик...,республика татарстан,республиканская спортивная школа олимпийского ...
